In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# import cudf
# %load_ext cudf.pandas
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import optuna
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Normalization, Dropout
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, RobustScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from sklearn.model_selection import train_test_split, KFold
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2024-07-24 21:12:07.860591: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 21:12:07.860701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 21:12:07.994655: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


In [2]:
import time
start_time = time.time()
train = pd.read_csv("/kaggle/input/playground-series-s4e7/train.csv", index_col=[0])
time_now = time.time() - start_time
print("train loading time is: ",time_now)
start_time = time.time()
test = pd.read_csv("/kaggle/input/playground-series-s4e7/test.csv", index_col=[0])
time_now = time.time() - start_time
print("test loading time is: ",time_now)

train loading time is:  21.165302991867065
test loading time is:  14.022817373275757


In [3]:
X = train.copy()
y = X.pop("Response")

In [4]:
# train.loc[X['Driving_License'] == 0]

In [5]:
X

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
id,,,,,,,,,,
0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187
1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288
2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254
3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76
4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294
...,...,...,...,...,...,...,...,...,...,...
11504793,Male,48,1,6.0,0,1-2 Year,Yes,27412.0,26.0,218
11504794,Female,26,1,36.0,0,< 1 Year,Yes,29509.0,152.0,115
11504795,Female,29,1,32.0,1,< 1 Year,No,2630.0,152.0,189


In [6]:
X.nunique()

Gender                      2
Age                        66
Driving_License             2
Region_Code                54
Previously_Insured          2
Vehicle_Age                 3
Vehicle_Damage              2
Annual_Premium          51728
Policy_Sales_Channel      152
Vintage                   290
dtype: int64

In [7]:
# cat_features = ["Gender", "Vehicle_Age", "Vehicle_Damage"]
# X[cat_features] = X[cat_features].astype('category')
# test[cat_features] = test[cat_features].astype('category')

In [8]:
# low_cardinality_features = ["Gender", "Driving_License", "Previously_Insured", "Vehicle_Age", "Vehicle_Damage"]
# X_one_hot = pd.get_dummies(X[low_cardinality_features])
# test_one_hot = pd.get_dummies(test[low_cardinality_features])

In [9]:
# high_cardinality_features = ["Region_Code", "Annual_Premium", "Policy_Sales_Channel", "Vintage"]

In [10]:
# X_high_cardinality = X[high_cardinality_features]
# test_high_cardinality = test[high_cardinality_features]
# X_high_cardinality

In [11]:
# X_new = pd.concat([X_one_hot,X_high_cardinality],axis=1)
# test_new = pd.concat([test_one_hot,test_high_cardinality],axis=1)

In [12]:
# X_new

In [13]:
# X_new.rename(columns={'Vehicle_Age_1-2 Year':'Vehicle_Age_betweeen_1and2Year','Vehicle_Age_< 1 Year':'Vehicle_Age_less_than_1Year','Vehicle_Age_> 2 Years':'Vehicle_Age_greater_than_2Year'}, inplace=True)
# test_new.rename(columns={'Vehicle_Age_1-2 Year':'Vehicle_Age_betweeen_1and2Year','Vehicle_Age_< 1 Year':'Vehicle_Age_less_than_1Year','Vehicle_Age_> 2 Years':'Vehicle_Age_greater_than_2Year'}, inplace=True)

In [14]:
# X_new

In [15]:
# ros = RandomOverSampler(random_state=42)
# X_ros, y_ros = ros.fit_resample(X,y)
# print("new shape is:", X_ros.shape)

In [16]:
X.dtypes

Gender                   object
Age                       int64
Driving_License           int64
Region_Code             float64
Previously_Insured        int64
Vehicle_Age              object
Vehicle_Damage           object
Annual_Premium          float64
Policy_Sales_Channel    float64
Vintage                   int64
dtype: object

In [17]:
def features_mapping(df):
    gender_mapping = {
        "Male":1,
        "Female":0
    }
    vehicle_age_mapping = {
    "< 1 Year":0,
    "1-2 Year":1,
    "> 2 Years":2
    }
    vehicle_damage_mapping = {
        "Yes":1,
        "No":0
    }
    
    df["Gender"] = df["Gender"].map(gender_mapping)
# X_ros["Gender"] = X_ros["Gender"].map(gender_mapping)
# X_rus["Gender"] = X_rus["Gender"].map(gender_mapping)
# X_train["Gender"] = X_train["Gender"].map(gender_mapping)
# X_valid["Gender"] = X_valid["Gender"].map(gender_mapping)

    df["Vehicle_Age"] = df["Vehicle_Age"].map(vehicle_age_mapping)
# X_ros["Vehicle_Age"] = X_ros["Vehicle_Age"].map(vehicle_age_mapping)
# X_rus["Vehicle_Age"] = X_rus["Vehicle_Age"].map(vehicle_age_mapping)
# X_train["Vehicle_Age"] = X_train["Vehicle_Age"].map(vehicle_age_mapping)
# X_valid["Vehicle_Age"] = X_valid["Vehicle_Age"].map(vehicle_age_mapping)

    df["Vehicle_Damage"] = df["Vehicle_Damage"].map(vehicle_damage_mapping)
# X_ros["Vehicle_Damage"] = X_ros["Vehicle_Damage"].map(vehicle_damage_mapping)
# X_rus["Vehicle_Damage"] = X_rus["Vehicle_Damage"].map(vehicle_damage_mapping)
    return df

def new_features(df):
#     df['Age_Annual_Premium'] = df['Age'] * df['Annual_Premium']
#     df['Vintage_Annual_Premium'] = df['Vintage'] * df['Annual_Premium']
    
#    df['VehicleDamage_RegionCode'] = df['Region_Code'] + df['Vehicle_Damage']
#     df = pd.get_dummies(df, columns=['VehicleDamage_RegionCode'], drop_first=True)
#     df['Vintage_Annual_Premium'] = df['Vintage'] * df['Annual_Premium']
    
#     df["Mean_Annual_Premium_by_region"] = df.groupby("Region_Code")["Annual_Premium"].transform('mean')
#     df["Damage_by_Region"] = df.groupby("Region_Code")["Annual_Premium"].transform('sum')
    
#     df['Premium_Age_Ratio'] = df["Annual_Premium"] / df["Age"]
#     df["Vintage_Age_Ratio"] = df["Vintage"] / df["Age"]
    
#     vip_thresh = df["Annual_Premium"].quantile(0.75)
#     df["High_Premium"] = (df["Annual_Premium"] > vip_thresh).astype('int')
#     df["Young_Driver"] = (df["Age"] < 25).astype('int')
#     df['Gender_VehicleAge'] = df['Gender'] + df['Vehicle_Age']
    df['DrivingLicense_VehicleAge'] = df['Driving_License'] + df['Vehicle_Age']
    df['DrivingLicense_VehicleDamage'] = df['Driving_License'] + df['Vehicle_Damage']
    df['DrivingLicense_PreviouslyInsured'] = df['Driving_License'] + df['Previously_Insured']
#     df = pd.get_dummies(df, columns=['Gender_Vehicle_Age'], drop_first=True)
    df['VehicleAge_VehicleDamage'] = df['Vehicle_Age']+ df['Vehicle_Damage']
#     df = pd.get_dummies(df, columns=["Vehicle_Age_Vehicle_Damage"], drop_first=True)

    df['PreviouslyInsured_VehicleDamage'] = df['Previously_Insured'] + df['Vehicle_Damage']
    df['PreviouslyInsured_VehicleAge'] = df['Previously_Insured'] + df['Vehicle_Age']
#     df['PreviouslyInsured_YoungDriver'] = df['Previously_Insured'].astype(str) + '_' + df['Young_Driver'].astype(str)
#     df = pd.get_dummies(df, columns=['PreviouslyInsured_YoungDriver'], drop_first=True)
    
#     df['YoungDriver_Vehicle_Damage'] = df['Young_Driver'].astype(str) + '_' + df['Vehicle_Damage'].astype(str)
#     df = pd.get_dummies(df, columns=['YoungDriver_Vehicle_Damage'], drop_first=True)
    return df

def memory_reduce(df):
    df['Gender'] = df['Gender'].astype('int8')
    df['Age'] = df['Age'].astype('int8')
    df['Driving_License'] = df['Driving_License'].astype('int8')
    df['Region_Code'] = df['Region_Code'].astype('float32')
    df['Previously_Insured'] = df['Previously_Insured'].astype('int8')
    df['Vehicle_Age'] = df['Vehicle_Age'].astype('int8')
    df['Vehicle_Damage'] = df['Vehicle_Damage'].astype('int8')
    df['Annual_Premium'] = df['Annual_Premium'].astype('float32')
    df['Policy_Sales_Channel'] = df['Policy_Sales_Channel'].astype('float32')
    df['Vintage'] = df['Vintage'].astype('int8')
    df['DrivingLicense_VehicleAge'] = df['DrivingLicense_VehicleAge'].astype('int8')
    df['DrivingLicense_VehicleDamage'] =  df['DrivingLicense_VehicleDamage'].astype('int8')
    df['DrivingLicense_PreviouslyInsured'] = df['DrivingLicense_PreviouslyInsured'].astype('int8')
    df['VehicleAge_VehicleDamage'] = df['VehicleAge_VehicleDamage'].astype('int8')
    df['PreviouslyInsured_VehicleDamage'] = df['PreviouslyInsured_VehicleDamage'].astype('int8')
    df['PreviouslyInsured_VehicleAge'] = df['PreviouslyInsured_VehicleAge'].astype('int8')
    return df

In [18]:
X = features_mapping(X)
X = new_features(X)
X = memory_reduce(X)

In [19]:
pd.set_option('display.max_columns',None)
X

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,DrivingLicense_VehicleAge,DrivingLicense_VehicleDamage,DrivingLicense_PreviouslyInsured,VehicleAge_VehicleDamage,PreviouslyInsured_VehicleDamage,PreviouslyInsured_VehicleAge
id,,,,,,,,,,,,,,,,
0,1,21,1,35.0,0,1,1,65101.0,124.0,-69,2,2,1,2,1,1
1,1,43,1,28.0,0,2,1,58911.0,26.0,32,3,2,1,3,1,2
2,0,25,1,14.0,1,0,0,38043.0,152.0,-2,1,1,2,0,1,1
3,0,35,1,1.0,0,1,1,2630.0,156.0,76,2,2,1,2,1,1
4,0,36,1,15.0,1,1,0,31951.0,152.0,38,2,1,2,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11504793,1,48,1,6.0,0,1,1,27412.0,26.0,-38,2,2,1,2,1,1
11504794,0,26,1,36.0,0,0,1,29509.0,152.0,115,1,2,1,1,1,0
11504795,0,29,1,32.0,1,0,0,2630.0,152.0,-67,1,1,2,0,1,1


In [20]:
# ros = RandomOverSampler(random_state=42)
# X_ros, y_ros = ros.fit_resample(X,y)
# print("new shape is:", X_ros.shape)

In [21]:
# del X
# del y

In [22]:
# train_ros = pd.concat([X_ros, y_ros], axis=1)
# train_ros, valid_ros = train_test_split(train_ros, test_size=0.35, stratify=train_ros["Response"], random_state=42)

# X_train = train_ros.copy()
# y_train = X_train.pop("Response")

# X_valid = valid_ros.copy()
# y_valid = X_valid.pop("Response")

In [23]:
# print(train_ros.shape)
# print("training shape: ",X_train.shape)
# print("validation shape: ",X_valid.shape)

In [24]:
# X_train = train_ros.copy()
# y_train = X_train.pop("Response")

# X_valid = valid_ros.copy()
# y_valid = X_valid.pop("Response")



In [25]:
# X_train = train_sub.copy()
# y_train = X_train.pop("Response")

# X_valid = valid_sub.copy()
# y_valid = X_valid.pop("Response")

In [26]:
# X_train

In [27]:
# X_train[["Gender","Vehicle_Age","Vehicle_Damage"]] = X_train[["Gender","Vehicle_Age","Vehicle_Damage"]].astype("category")
# X_valid[["Gender","Vehicle_Age","Vehicle_Damage"]] = X_valid[["Gender","Vehicle_Age","Vehicle_Damage"]].astype("category")

In [28]:
# gender_mapping = {
#     "Male":1,
#     "Female":0
# }
# vehicle_age_mapping = {
#     "< 1 Year":0,
#     "1-2 Year":1,
#     "> 2 Years":2
# }
# vehicle_damage_mapping = {
#     "Yes":1,
#     "No":0
# }

# X["Gender"] = X["Gender"].map(gender_mapping)
# # X_ros["Gender"] = X_ros["Gender"].map(gender_mapping)
# X_rus["Gender"] = X_rus["Gender"].map(gender_mapping)
# # X_train["Gender"] = X_train["Gender"].map(gender_mapping)
# # X_valid["Gender"] = X_valid["Gender"].map(gender_mapping)

# X["Vehicle_Age"] = X["Vehicle_Age"].map(vehicle_age_mapping)
# # X_ros["Vehicle_Age"] = X_ros["Vehicle_Age"].map(vehicle_age_mapping)
# X_rus["Vehicle_Age"] = X_rus["Vehicle_Age"].map(vehicle_age_mapping)
# # X_train["Vehicle_Age"] = X_train["Vehicle_Age"].map(vehicle_age_mapping)
# # X_valid["Vehicle_Age"] = X_valid["Vehicle_Age"].map(vehicle_age_mapping)

# X["Vehicle_Damage"] = X["Vehicle_Damage"].map(vehicle_damage_mapping)
# # X_ros["Vehicle_Damage"] = X_ros["Vehicle_Damage"].map(vehicle_damage_mapping)
# X_rus["Vehicle_Damage"] = X_rus["Vehicle_Damage"].map(vehicle_damage_mapping)
# # X_train["Vehicle_Damage"] = X_train["Vehicle_Damage"].map(vehicle_damage_mapping)
# # X_valid["Vehicle_Damage"] = X_valid["Vehicle_Damage"].map(vehicle_damage_mapping)

In [29]:
# smote = SMOTE(random_state=42)

# X_smoted, y_smoted = smote.fit_resample(X, y)
# train_smoted = pd.concat([X_smoted, y_smoted], axis=1)

In [30]:
test = features_mapping(test)
test = new_features(test)
test = memory_reduce(test)

In [31]:
# features_to_scale = ['Age','Vintage','Annual_Premium',	'Policy_Sales_Channel',	'Age_Annual_Premium',	'Vintage_Annual_Premium',	'Mean_Annual_Premium_by_region',	'Damage_by_Region',	'Premium_Age_Ratio',	'Vintage_Age_Ratio']
# features_to_scale = ['Age', 'Vintage', 'Annual_Premium', 'Policy_Sales_Channel']
# scaler = RobustScaler()
# X[features_to_scale] = scaler.fit_transform(X[features_to_scale])

In [32]:
# test[features_to_scale] = scaler.transform(test[features_to_scale])

In [33]:
# pd.reset_option('all')

In [34]:
# X

In [35]:
# test["Gender"] = test["Gender"].map(gender_mapping)
# # X_ros["Gender"] = X_ros["Gender"].map(gender_mapping)
# test["Vehicle_Age"] = test["Vehicle_Age"].map(vehicle_age_mapping)
# # X_ros["Vehicle_Age"] = X_ros["Vehicle_Age"].map(vehicle_age_mapping)
# test["Vehicle_Damage"] = test["Vehicle_Damage"].map(vehicle_damage_mapping)
# # X_ros["Vehicle_Damage"] = X_ros["Vehicle_Damage"].map(vehicle_damage_mapping)

In [36]:
continuous_features = ["Age","Region_Code","Annual_Premium","Policy_Sales_Channel","Vintage"]
# scaler = StandardScaler()
scaler = RobustScaler()
X[continuous_features] = scaler.fit_transform(X[continuous_features])
test[continuous_features] = scaler.fit_transform(test[continuous_features])

In [37]:
# test[continuous_features] = scaler.transform(test[continuous_features])

In [38]:
# lgbm = LGBMClassifier(verbose=0, n_jobs=-1)
# lgbm.fit(X_ros, y_ros)

In [39]:
# feature_importances = lgbm.feature_importances_

# feature_importance_df = pd.DataFrame({'feature': X.columns, 'importance': feature_importances})

# feature_importance_df = feature_importance_df.sort_values('importance', ascending=False)

# print(feature_importance_df.head(10))

In [40]:
# X_ros.shape

In [41]:
# y_ros.shape

In [42]:
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [43]:
# del X
# del y

In [44]:
# del X_ros, y_ros

In [45]:
# X_ros

In [46]:
class_weights = compute_class_weight('balanced', classes = np.unique(y), y=y)
class_labels = np.unique(y)
class_weights_dict = {class_labels[i]: class_weights[i] for i in range(len(class_weights))}

In [47]:
sample_weights = np.where(y==0, class_weights_dict[0], class_weights_dict[1],)

In [48]:
sample_weights.shape

(11504798,)

In [49]:
# # Compile the model
# model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=0.55), metrics=['accuracy', keras.metrics.AUC()])

# # Print the model summary
# model.summary()

In [50]:
# history = model.fit(X_train,y_train,
#                    epochs=100,
#                    batch_size=10000,
#                    sample_weight=sample_weights,
#                    validation_data=(X_valid,y_valid))

In [51]:
# model = Sequential([
#     Dense(128, activation='mish'),
#     layers.LayerNormalization(),
#     Dense(64, activation='mish'),
#     layers.LayerNormalization(),
#     Dense(32, activation='mish'),
#     layers.LayerNormalization(),
#     Dense(16, activation='mish'),
#     layers.LayerNormalization(),
#     Dense(8, activation='mish'),
#     layers.LayerNormalization(),
#     Dense(1, activation='sigmoid')
# ])

# model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC()])

# model.summary()

In [52]:
# y_train.shape

In [53]:
# history = model.fit(X_train,y_train,
#                    epochs=50,
#                    batch_size=10000,
#                    sample_weight=sample_weights,
#                    validation_data=(X_valid,y_valid))

In [54]:
# lgbm = LGBMClassifier(device='gpu', n_jobs=-1)
# lgbm.fit(X_ros, y_ros)

In [55]:
xgb_params = {              
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'booster': 'gbtree',
    'n_estimators': 1500,       
    'learning_rate': 0.55,                   
    'min_child_weight': 17,      
    'reg_lambda': 0.2,           
    'reg_alpha': 7,              
    'max_bin': 52000,            
    'colsample_bytree': 0.65,    
    'max_delta_step': 2,         
    'random_state': 0,
    'tree_method': 'hist',
    'device': 'cuda',
    'n_jobs': -1,
    'scale_pos_weight': class_weights_dict[1] 
}
lgbm_params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'num_boost_round': 1500,
    'learning_rate': 0.55,
    'min_child_samples': 17,
    'lambda_l2': 0.2,
    'lambda_l1': 7,
    'max_bin': 255,
    'feature_fraction': 0.65,
    'random_state': 0,
    'tree_learner': 'serial',  # 'serial' for CPU; 'data' or 'feature' for parallel learning
    'device_type': 'gpu',  # Use 'gpu' if using GPU, else 'cpu'
    'num_threads': -1,
    'class_weight': 'balanced'
}
cat_params = {
    'objective': 'Logloss',
    'eval_metric': 'AUC',
    'iterations': 1500,
    'learning_rate': 0.55,
    'depth': 6,  # Depth is analogous to max_depth in other GBMs; you might need to tune it
    'l2_leaf_reg': 0.2,  # Analogous to reg_lambda
    'random_seed': 0,
    'task_type': 'GPU',  # 'GPU' for GPU training; 'CPU' for CPU
    'thread_count': -1,
    'class_weights': [class_weights_dict[0],class_weights_dict[1]]
}

In [56]:
# models = {
# #     "HistGradientBoosting": HistGradientBoostingClassifier(class_weight='balanced'),
# #     "LightGBM": LGBMClassifier(device='gpu', n_jobs=-1,class_weight= 'balanced'),
# #     "CatBoost": CatBoostClassifier(task_type='GPU', thread_count=-1, class_weights= [class_weights_dict[0], class_weights_dict[1]]),
#       "XGBoost": XGBClassifier(**xgb_params), # attained 0.8899 score
# #     'Lightgbm': LGBMClassifier(**lgbm_params),
# #     'Catboost': CatBoostClassifier(**cat_params)
# }

# # KFold cross-validation
# kf = KFold(n_splits=3, shuffle=True, random_state=1)

# # Dictionary to store results
# roc_auc_results = {}
# logloss_results = {}

# X = X.reset_index(drop=True)
# y = y.reset_index(drop=True)
# # Evaluate each model
# for name, model in models.items():
#     roc_auc_list = []
#     logloss_list = []
#     for train_index, test_index in kf.split(X):
#         X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#         y_train, y_test = y.iloc[train_index], y.iloc[test_index]
#         model.fit(X_train, y_train)
#         preds = model.predict_proba(X_test)[:,1]
#         roc_auc = roc_auc_score(y_test, preds)
#         roc_auc_list.append(roc_auc)
#     roc_auc_results[name] = roc_auc_list
#     print(f"{name} - ROC AUC: {np.mean(roc_auc_list):.4f} (+/- {np.std(roc_auc_list):.4f})")

In [57]:
# X_ros = X_ros.reset_index(drop=True)
# y_ros = y_ros.reset_index(drop=True)

# # Print indices
# print(f"X_ros index:\n{X_ros.index}")
# print(f"y_ros index:\n{y_ros.index}")

# # Evaluate each model
# for fold, (train_index, test_index) in enumerate(kf.split(X_ros), 1):
#     print(f"\nFold {fold}:")
#     print(f"Train indices: {train_index}")
#     print(f"Test indices: {test_index}")

In [58]:
# roc_auc_results

In [59]:
# # Display summary of ROC AUC results
# roc_auc_df = pd.DataFrame(roc_auc_results).T
# roc_auc_df.columns = ['Fold 1', 'Fold 2', 'Fold 3']
# roc_auc_df['Mean ROC AUC'] = roc_auc_df.mean(axis=1)
# roc_auc_df['Std Dev'] = roc_auc_df.std(axis=1)

# print("\nSummary of ROC AUC results:")
# print(roc_auc_df)

# # Display summary of log loss results
# # logloss_df = pd.DataFrame(logloss_results).T
# # logloss_df.columns = ['Fold 1', 'Fold 2', 'Fold 3']
# # logloss_df['Mean Log Loss'] = logloss_df.mean(axis=1)
# # logloss_df['Std Dev'] = logloss_df.std(axis=1)

# # print("\nSummary of LogLoss results:")
# # print(logloss_df)

In [60]:
# Summary of ROC AUC results:
#                         Fold 1    Fold 2    Fold 3  Mean ROC AUC   Std Dev
# RandomForest          0.851484  0.852166  0.851353      0.851668  0.000356
# AdaBoost              0.859190  0.859731  0.858918      0.859280  0.000338
# GradientBoosting      0.865154  0.865599  0.864776      0.865176  0.000336
# HistGradientBoosting  0.875041  0.875562  0.875206      0.875270  0.000218
# ExtraTrees            0.841332  0.841713  0.841180      0.841408  0.000224
# LightGBM              0.875722  0.876071  0.875340      0.875711  0.000298
# CatBoost              0.870625  0.871408  0.870140      0.870724  0.000523
# XGBoost               0.878114  0.878968  0.877698      0.878260  0.000528

# Summary of LogLoss results:
#                         Fold 1    Fold 2    Fold 3  Mean Log Loss   Std Dev
# RandomForest          0.475906  0.474242  0.475362       0.475170  0.000693
# AdaBoost              0.673588  0.673637  0.673632       0.673619  0.000022
# GradientBoosting      0.413304  0.412833  0.413254       0.413130  0.000211
# HistGradientBoosting  0.401352  0.400915  0.400952       0.401073  0.000198
# ExtraTrees            0.647964  0.648817  0.657455       0.651412  0.004287
# LightGBM              0.400750  0.400494  0.400831       0.400692  0.000143
# CatBoost              0.405725  0.405028  0.405738       0.405497  0.000332
# XGBoost               0.397448  0.396583  0.397677       0.397236  0.000471


# summary of roc_auc results with no random over sampling with feature engineering
# Summary of ROC AUC results:
#                         Fold 1    Fold 2    Fold 3  Mean ROC AUC   Std Dev
# HistGradientBoosting  0.875687  0.875219  0.875401      0.875436  0.000193
# LightGBM              0.875538  0.875763  0.876056      0.875785  0.000212
# CatBoost              0.870475  0.870682  0.870775      0.870644  0.000125
# XGBoost               0.877918  0.878111  0.878382      0.878137  0.000191

In [61]:
# import matplotlib.pyplot as plt

# save_dir = "/kaggle/working/feature_importance_plots"
# os.makedirs(save_dir, exist_ok=True)

# for name, model in models.items():
#     plt.figure(figsize=(10, 5))
    
#     if hasattr(model, 'feature_importances_'):
#         # For models that have feature_importances_ attribute (like RandomForestClassifier, ExtraTreesClassifier, etc.)
#         feature_importances = model.feature_importances_
#         plt.barh(X.columns, feature_importances)
#         plt.title(f"Feature importances for {name}")
#         plt.xlabel("Importance")
#         plt.ylabel("Feature")
#         plt.tight_layout()
        
#         # Save figure
#         save_path = os.path.join(save_dir, f"{name}_feature_importance.png")
#         plt.savefig(save_path)
        
#         # Display figure
#         plt.show()
        
#     elif hasattr(model, 'plot_feature_importances'):
#         # For HistGradientBoostingClassifier, CatBoostClassifier, etc. that provide plot_feature_importances() method
#         model.plot_feature_importances()
#         plt.title(f"Feature importances for {name}")
#         plt.tight_layout()
        
#         # Save figure
#         save_path = os.path.join(save_dir, f"{name}_feature_importance.png")
#         plt.savefig(save_path)
        
#         # Display figure
#         plt.show()
        
#     else:
#         print(f"{name} does not have feature importances available.")

In [62]:
def objective(trial):
    # Define the hyperparameter search space
    param = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'booster': 'gbtree',
        'n_estimators': trial.suggest_int('n_estimators', 1000, 2000),       
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.55),                   
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),      
        'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 10.0),           
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10.0),              
        'max_bin': trial.suggest_int('max_bin', 100, 60000),            
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),    
        'max_delta_step': trial.suggest_int('max_delta_step', 0, 10),         
        'random_state': 0,
        'tree_method': 'hist',
        'device': 'cuda',
        'n_jobs': -1,
        'scale_pos_weight': class_weights_dict[1] 
    }
    
    # Split data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
    
    # Train the model
    model = XGBClassifier(**param)
    model.fit(X_train, y_train)
    
    # Predict probabilities
    y_pred = model.predict_proba(X_val)[:, 1]
    
    # Evaluate the model
    auc = roc_auc_score(y_val, y_pred)
    
    return auc

# Create the study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # You can adjust the number of trials

# Retrieve the best hyperparameters
best_params = study.best_params
print("Best hyperparameters: ", best_params)

# Train the final model with the best hyperparameters
best_model = XGBClassifier(**best_params)
best_model.fit(X, y)

[I 2024-07-24 21:13:22,191] A new study created in memory with name: no-name-6a002344-c278-42b0-a15f-02bc586eb800
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [21:19:28] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
[I 2024-07-24 21:19:30,670] Trial 0 finished with value: 0.88813615502404 and parameters: {'n_estimators': 1505, 'learning_rate': 0.2940142634640344, 'min_child_weight': 6, 'reg_lambda': 6.118091977503622, 'reg_alpha': 1.9465718434518036, 'max_bin': 42691, 'colsample_bytree': 0.3566033428647203, 'max_delta_step': 7}. Best is tri

Best hyperparameters:  {'n_estimators': 1924, 'learning_rate': 0.31581996209523133, 'min_child_weight': 11, 'reg_lambda': 3.2276536566141476, 'reg_alpha': 3.457150872709881, 'max_bin': 59883, 'colsample_bytree': 0.7110267255919047, 'max_delta_step': 10}


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7110267255919047, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.31581996209523133,
              max_bin=59883, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=10, max_depth=None, max_leaves=None,
              min_child_weight=11, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1924, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [63]:
# dict1 = {
#     "a":1,
#     "b":2,
#     "c":3,
#     "d":4
# }
# dict2 = {}
# list2 = [1,2,3,4]
# for key, value in dict1.items():
#     list1 = []
#     for i in range(4):
#         list1.append(list2[i])
#     dict2[key] = list1
    
# print(dict2)

In [64]:
new_test = pd.read_csv("/kaggle/input/playground-series-s4e7/test.csv")
idx = new_test["id"]

In [65]:
# lgbm_preds

In [66]:
# xgb = XGBClassifier(tree_method='gpu_hist', n_jobs=-1)
# xgb.fit(X_ros, y_ros)

In [67]:
xgb_preds = best_model.predict_proba(test)[:,1]

In [68]:
result = pd.DataFrame({
    "id":idx,
    "Response":xgb_preds
})
result.to_csv("submission.csv", index=False)

In [69]:
# roc_score = roc_auc_score(y_valid, lgbm_preds)
# print(roc_score)

In [70]:
# hgbc = HistaGradientBoostingClassifier(n_jobs=-1)
# hgbc.fit(X, y)